In [15]:
import random

import pandas as pd
from rapidfuzz import fuzz as rapid_fuzz
from rapidfuzz.process import cdist
from thefuzz import fuzz

In [20]:
def generate_ids():
    """Generate ID-like strings."""
    while True:
        country_code = "NLD"
        gender = random.choice(["M", "F"])
        first = f"{random.randint(0, 9999999):07d}"
        second = f"{random.randint(0, 9999999999999999):016d}"

        yield f"{country_code}{first}{gender}{second}"


def similarity(target_id: str, search_ids: pd.Series) -> pd.Series:
    """Compute Levenhstein distance between IDs."""
    return search_ids.map(lambda search_id: fuzz.ratio(target_id, search_id))


def rapid_similarity(target_id: str, search_ids: pd.Series) -> pd.Series:
    """Compute Levenhstein distance between IDs."""
    return pd.Series(
        cdist([target_id], search_ids, scorer=rapid_fuzz.ratio, workers=-1)[0]
    )


def equality(target_id: str, search_ids: pd.Series) -> pd.Series:
    """Compare an ID to a list of IDs."""
    return search_ids.map(lambda search_id: search_id == target_id)

In [5]:
id_generator = generate_ids()

In [6]:
n = 5_000_000
ids = pd.Series([next(id_generator) for _ in range(n)])
ids.sample(5)

3965952    NLD7862985M9584249290578852
2219008    NLD2620976M3543559410513773
1392104    NLD4190742F2322235162041885
1109549    NLD1127711F7430815707386695
2412081    NLD0493995M0682563977057378
dtype: object

In [21]:
rapid_similarity(ids[0], ids).sort_values(ascending=False)

0          100.000000
2808724     74.074074
749630      70.370369
4055081     70.370369
3743367     70.370369
              ...    
4162008     25.925926
4270757     25.925926
4324848     25.925926
4320376     25.925926
4435688     25.925926
Length: 5000000, dtype: float32

In [22]:
%timeit similarity(ids[0], ids)

5.3 s ± 129 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [24]:
%timeit rapid_similarity(ids[0], ids)

834 ms ± 16.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%timeit equality(ids[0], ids)

1.04 s ± 16.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
